In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/MyDrive/ETH/AML/task1

/content/gdrive/MyDrive/ETH/AML/task1


In [3]:
import numpy as np
import pandas as pd

In [4]:
train_cvsx=pd.read_csv('X_train.csv',dtype=np.double)
train_x=train_cvsx.iloc[:,1:]


In [5]:
train_cvsx.iloc[0:3]

,id,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,...,x792,x793,x794,x795,x796,x797,x798,x799,x800,x801,x802,x803,x804,x805,x806,x807,x808,x809,x810,x811,x812,x813,x814,x815,x816,x817,x818,x819,x820,x821,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
0,0.0,10.891876,832442.812375,20585.544083,1028.369495,1.163780e+06,9.199135,597900.477629,NaN,1.144294e+06,785176.201298,38042.008654,106.644316,67475.900839,944696.306555,895.088214,10.819457,155995.454645,10.114602,98.776154,90.200020,107.176656,1.232961e+06,2538.045240,1.038292e+06,10.445353,1530.001316,10403.207060,101.974225,9902.072478,10.675578,1.965179,108.095109,10.491893,985.743949,139494.789412,5555.414472,10079.665710,NaN,1199.950457,...,2.586985,107835.473217,1.050203e+06,961.000963,102.726565,11230.077183,1132.878438,10955.849718,2.358928,1.079014e+06,10.271036,309702.424398,1074.952720,10083.165942,106208.115341,102.072379,119.507651,NaN,1077.836958,NaN,87766.029819,1857.070822,96.997544,NaN,1011.959739,89.184535,107577.830266,2.139276,-3163.219791,1057.888075,1.024198e+06,-855.549602,12176.073427,10.647729,10.916371,1220.065443,8.566724,1.036263e+06,85338.558539,103088.664210
1,1.0,11.512994,832442.898114,NaN,1012.624877,1.028911e+06,10.906408,597900.458612,8127.016078,1.099166e+06,785176.258299,50949.420639,NaN,93179.779900,926043.432843,1159.073327,10.403125,NaN,10.365511,104.569049,88.722820,102.703415,NaN,2033.004526,8.321971e+05,10.503675,2119.097861,10700.279888,101.354911,10414.075263,10.860626,1.716548,102.045514,10.073451,880.965682,NaN,4784.223551,10968.228448,NaN,937.225484,...,1.796449,104811.157714,1.091030e+06,1052.815245,82.458489,10487.069231,695.406075,10985.255804,1.886463,1.072502e+06,8.178066,557053.464687,1060.352954,10458.065277,108798.115287,100.207625,NaN,95559.700238,1028.505709,212339.931756,99387.412802,1818.087105,101.159059,1073.745683,1017.273256,NaN,NaN,1.746126,-2661.694233,1025.245447,1.086806e+06,-787.397942,10493.095660,10.586492,9.463962,917.094909,10.231822,1.007163e+06,95695.020645,105161.109422
2,2.0,11.052185,832442.896307,20585.512844,1003.953827,9.231756e+05,9.212979,597900.426764,10738.092422,1.027863e+06,785176.223468,47259.321206,103.365571,69377.195188,828900.306175,NaN,10.667846,155995.367237,10.437764,94.987987,101.695502,100.034049,NaN,2138.054129,6.792212e+05,10.967175,1926.041068,10633.374493,107.429192,12163.033117,10.581398,2.340156,101.247931,10.782089,NaN,139494.776583,4317.864410,10628.203050,10381.020335,977.000200,...,2.932762,101814.683255,1.001865e+06,NaN,96.292749,11122.087230,862.889649,10308.209427,2.406808,1.048036e+06,NaN,396496.091177,1004.185813,10249.487036,NaN,106.147877,94.671834,91827.987615,1061.749457,212339.935348,108348.187401,1895.067520,NaN,1015.708474,903.520905,100.032696,108205.355183,2.421048,-2671.968203,1039.024306,1.018533e+06,-906.997242,10959.516944,10.769287,10.342160,637.027802,10.705461,1.019955e+06,80253.299882,104177.051666


In [6]:
train_x.iloc[0:3]

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,...,x792,x793,x794,x795,x796,x797,x798,x799,x800,x801,x802,x803,x804,x805,x806,x807,x808,x809,x810,x811,x812,x813,x814,x815,x816,x817,x818,x819,x820,x821,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
0,10.891876,832442.812375,20585.544083,1028.369495,1.163780e+06,9.199135,597900.477629,NaN,1.144294e+06,785176.201298,38042.008654,106.644316,67475.900839,944696.306555,895.088214,10.819457,155995.454645,10.114602,98.776154,90.200020,107.176656,1.232961e+06,2538.045240,1.038292e+06,10.445353,1530.001316,10403.207060,101.974225,9902.072478,10.675578,1.965179,108.095109,10.491893,985.743949,139494.789412,5555.414472,10079.665710,NaN,1199.950457,14709.216512,...,2.586985,107835.473217,1.050203e+06,961.000963,102.726565,11230.077183,1132.878438,10955.849718,2.358928,1.079014e+06,10.271036,309702.424398,1074.952720,10083.165942,106208.115341,102.072379,119.507651,NaN,1077.836958,NaN,87766.029819,1857.070822,96.997544,NaN,1011.959739,89.184535,107577.830266,2.139276,-3163.219791,1057.888075,1.024198e+06,-855.549602,12176.073427,10.647729,10.916371,1220.065443,8.566724,1.036263e+06,85338.558539,103088.664210
1,11.512994,832442.898114,NaN,1012.624877,1.028911e+06,10.906408,597900.458612,8127.016078,1.099166e+06,785176.258299,50949.420639,NaN,93179.779900,926043.432843,1159.073327,10.403125,NaN,10.365511,104.569049,88.722820,102.703415,NaN,2033.004526,8.321971e+05,10.503675,2119.097861,10700.279888,101.354911,10414.075263,10.860626,1.716548,102.045514,10.073451,880.965682,NaN,4784.223551,10968.228448,NaN,937.225484,12616.740072,...,1.796449,104811.157714,1.091030e+06,1052.815245,82.458489,10487.069231,695.406075,10985.255804,1.886463,1.072502e+06,8.178066,557053.464687,1060.352954,10458.065277,108798.115287,100.207625,NaN,95559.700238,1028.505709,212339.931756,99387.412802,1818.087105,101.159059,1073.745683,1017.273256,NaN,NaN,1.746126,-2661.694233,1025.245447,1.086806e+06,-787.397942,10493.095660,10.586492,9.463962,917.094909,10.231822,1.007163e+06,95695.020645,105161.109422
2,11.052185,832442.896307,20585.512844,1003.953827,9.231756e+05,9.212979,597900.426764,10738.092422,1.027863e+06,785176.223468,47259.321206,103.365571,69377.195188,828900.306175,NaN,10.667846,155995.367237,10.437764,94.987987,101.695502,100.034049,NaN,2138.054129,6.792212e+05,10.967175,1926.041068,10633.374493,107.429192,12163.033117,10.581398,2.340156,101.247931,10.782089,NaN,139494.776583,4317.864410,10628.203050,10381.020335,977.000200,13070.565105,...,2.932762,101814.683255,1.001865e+06,NaN,96.292749,11122.087230,862.889649,10308.209427,2.406808,1.048036e+06,NaN,396496.091177,1004.185813,10249.487036,NaN,106.147877,94.671834,91827.987615,1061.749457,212339.935348,108348.187401,1895.067520,NaN,1015.708474,903.520905,100.032696,108205.355183,2.421048,-2671.968203,1039.024306,1.018533e+06,-906.997242,10959.516944,10.769287,10.342160,637.027802,10.705461,1.019955e+06,80253.299882,104177.051666


In [7]:
train_cvsy=pd.read_csv('y_train.csv',dtype=np.double)
train_y=train_cvsy.iloc[:,1:]

In [8]:
train_y.iloc[0:3]

,y
0,71.0
1,73.0
2,66.0


###preprocess
  #1.imputation of missing values
  ###scikit-learn KNN imputer
  #potential problem : sensitive to outlier
  # parameter adjust n_neighbors= xxx try  different # could 

In [9]:
from sklearn.impute import KNNImputer

In [10]:

imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
train_xp=imputer.fit_transform(train_x)


In [11]:
train_xp=pd.DataFrame(train_xp)

In [ ]:
#df.isnull().sum()

#Outlier Detection
## isolation forest
 max_features, contamination, could try to reset 

you’ll generate the biggest performance boost by using a grid search technique with cross validation #todo

In [12]:
from sklearn.ensemble import IsolationForest

In [13]:
isofore=IsolationForest(n_estimators=50,max_features=10,max_samples='auto')
train_xppl=isofore.fit_predict(train_xp)
train_xpp=train_xp[(train_xppl!=-1)]
train_ypp=train_y[(train_xppl!=-1)]

In [14]:
train_ypp.shape

(1204, 1)

In [15]:
train_xpp.shape

(1204, 832)

In [16]:
train_y.shape

(1212, 1)

simluate train test split



In [17]:
xtrain=train_xpp.iloc[:700,:]
ytrain=train_ypp.iloc[:700,:]
xtest=train_xpp.iloc[700:1000,:]
ytest=train_ypp.iloc[700:1000,:]

#Feature selection 
As F-test captures only linear dependency, it rates x_1 as the most discriminative feature. On the other hand, mutual information can capture any kind of dependency between variables and it rates x_2 as the most discriminative feature, which probably agrees better with our intuitive perception for this example. Both methods correctly marks x_3 as irrelevant.


In [18]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression

In [19]:
selector = SelectKBest(score_func=mutual_info_regression, k=100)
selector.fit(xtrain,np.ravel(ytrain))
train_xppp=selector.transform(xtrain)
test_xp1p3=selector.transform(xtest)


In [20]:
test_xp1p3.shape

(300, 100)

 To identify the selected features we use get_support() function and filter out them from the features name list.

In [ ]:
filter = selector.get_support()
 

Regression


In [21]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF,DotProduct, ConstantKernel,WhiteKernel,Matern,RationalQuadratic

In [ ]:
## simulate test & train 

In [38]:
kernel=RBF()+WhiteKernel()
gpr=gpr = GaussianProcessRegressor(kernel=kernel,alpha=1e-10, n_restarts_optimizer=2, normalize_y=True, copy_X_train=False, random_state=None)
#gpr.fit(train_xppp, ytrain)
gpr.fit(xtrain,ytrain)
#ypred=gpr.predict(test_xp1p3)
ypred=gpr.predict(xtest)

evaluation

In [32]:
from sklearn.metrics import r2_score

In [37]:
score=r2_score(ytest,ypred)
print(score)

-5.772605966658517e-05


In [ ]:
### test
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_xp, train_y, test_size=0.3, random_state=1)
# scale the data using StandardScaler() so the various column values are on an even scale.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#X_train.shape

(848, 832)

grid search tuning hyperparameter

In [ ]:
model = IsolationForest(random_state=47)

param_grid = {'n_estimators': [1000, 1500], 
              'max_samples': [10], 
              'contamination': ['auto', 0.0001, 0.0002], 
              'max_features': [10, 15], 
              'bootstrap': [True], 
              'n_jobs': [-1]}

grid_search = model_selection.GridSearchCV(model, 
                                           param_grid,
                                           scoring="neg_mean_squared_error", 
                                           refit=True,
                                           cv=10, 
                                           return_train_score=True)
grid_search.fit(X_train, y_train)

best_model = grid_search.fit(X_train, y_train)
print('Optimum parameters', best_model.best_params_)